In [2]:
%load_ext watermark
%watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
2017-09-16T18:40:39+02:00

CPython 3.6.1
IPython 5.3.0

compiler   : GCC 4.8.2 20140120 (Red Hat 4.8.2-15)
system     : Linux
release    : 4.10.0-33-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


In [3]:
from sklearn import datasets

In [4]:
boston = datasets.load_boston()

In [5]:
from sklearn.linear_model import LinearRegression

In [6]:
model = LinearRegression()

In [7]:
model.fit(X=boston["data"], y=boston["target"])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [8]:
y_objetivo = boston["target"]
y_pred = model.predict(boston["data"])

In [9]:
from sklearn import metrics

**Error Absoluto Medio**

El Error Absoluto Medio (Mean Absolute Error o MAE) se define como:

$$\frac{1}{n}\sum_{i=1}^{n}|y_i -\hat{y}_i|$$

Es decir, la media de las diferencias entre la varible objetivo y las predicciones sin el signo.

MAE es una métrica robusta, en cuanto a que no varia mucho si hay valores extremos en los datos. El error se puede interpretar como unidades de la variable objetivo (por ejemplo, si la variable objetivo es en dolares MAE estará también en dólares).

In [10]:
metrics.mean_absolute_error?

In [11]:
metrics.mean_absolute_error(y_objetivo, y_pred)

3.2729446379969378

**Error Cuadrático Medio**

El Error cuadrático Medio (Mean Squared Error o MSE)

$$\frac{1}{n}\sum_{i=1}^{n}(y_i -\hat{y}_i)^2$$

Dado que el MSE se define en unidades al cuadrado, lo cual no es intuitivo (¿dolares cuadrados?), generalmente se usa su raíz.

**Raíz del Error Cuadrático Medio**

La Raíz del Error Cuadrático Medio (Root Mean Squared Error o RMSE) se diferencia del MSE en que el resultado se puede medir en las mismas unidades que la variable objetivo

$$\sqrt{\frac{1}{n}\sum_{i=1}^{n}(y_i -\hat{y}_i)^2}$$



Sin embargo, tiene un problema y es que da más importancia a los errores grandes. Por ejemplo en el Boston Housing Dataset, si tenemos dos observaciones y sus predicciones:

```
observacion1: MEDV: 10  MEDV_pred: 15    RMSE: (10-15)²=25
observacion2: MEDV: 1000 MEDV_pred: 1010 RMSE: (1000-1010)²=100 
```

El usar RMSE como medida de error significa que se le dará mas peso al error de la observacion2 que al de la observación1, pese a que un error de 5,000 dólares en una zona donde el valor medio es 10,000$ es un error mucho mas grave que un error de 10,000 dólares  en una zona donde el valor medio de las casas es de 1 millon de dólares.

RMSE es más sensible que MAE a variaciones en los errores de predicción, aquí tenemos un ejemplo donde se ve esto.

![mae_vs_mse](https://cdn-images-1.medium.com/max/1371/1*YTxb8K2XZIisC944v6rERw.png)

En este ejemplo se ve como para los tres casos, el MAE no varía, mientras que en función de la distribución de errores el RMSE puede ser igual al MAE (todos los errores iguales) o mucho mayor (muchos errores nulos con un error muy grande)

Para aquellos casos donde lo que queramos es esto (evitar errores grandes) y cuando la distribución de la variable objetivo este bién distribuida se puede usar RMSE en vez de MAE.

In [12]:
import numpy as np

np.sqrt(metrics.mean_squared_error(y_objetivo, y_pred))

4.6795063006355164

**Coeficiente de determinación**

El coeficiente de determinación (R2 o R-squared) mide la porción de la varianza de la variable objetivo que se puede explicar por el modelo.

R2 tiene un valor máximo de 1 (cuando el modelo explica toda la varianza) , aunque puede tener valores negativos.

Hay [varias formas de definir R2](https://en.wikipedia.org/wiki/Coefficient_of_determination) , pero una de las más sencillas es simplemente la correlacion (definida como la [Correlación de Pearson](https://es.wikipedia.org/wiki/Coeficiente_de_correlaci%C3%B3n_de_Pearson)) entre la variable objetivo y las predicciones , elevada al cuadrado.

![pmcc](https://wikimedia.org/api/rest_v1/media/math/render/svg/01d103c10e6d4f477953a9b48c69d19a954d978a)

Un problema importante que tiene R2 es que no nos indica si el modelo explica la varianza debido a que está sobreajustado (overfitted). Por eso una medida mejor es el Coeficiente de Determinacion ajustado (Adjusted R-squared), que tiene en consideracion la complejidad del modelo.

$$1 - \frac{(1 - R^2)(n-1)}{(n-k-1)}$$

donde `n` es el número de observaciones y `k` es el numero de coeficientes del modelo (sin contar el término independiente)

In [13]:
modelo_r2 = metrics.r2_score(y_objetivo, y_pred)
modelo_r2

0.74060774286494269

In [14]:
import numpy
np.corrcoef(y_objetivo, y_pred)**2

array([[ 1.        ,  0.74060774],
       [ 0.74060774,  1.        ]])

In [15]:
len(model.coef_)

13

In [16]:
modelo_r2_ajustado = 1 - (1-modelo_r2)*(len(boston["target"])-1)/  \
                         (len(boston["target"])-boston["data"].shape[1]-1)
modelo_r2_ajustado

0.73375388241218709